Arctic sea ice forecasting using deep neural networks.

In [1]:
import numpy as np
from pathlib import Path
import sklearn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional
import xarray as xr

/home/yangliu/miniconda3/envs/ml/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load and prepare data.

In [2]:
# path
path_data = Path("/home/yangliu/MLexpo/data")
erai_path = path_data / "ERA-Interim"
oras_path = path_data / "ORAS4"

In [3]:
start_year = 2000
end_year = 2017
# number of weeks per year
nweeks = 48
# load data
sic_xr = xr.load_dataset(erai_path / "sic_weekly_erai_1979_2017.nc")
sic_xr = sic_xr.sel(year=slice(start_year, end_year), latitude=slice(81, 63),
 longitude=slice(18, 59.5))
sic_xr

<xarray.Dataset>
Dimensions:    (year: 18, week: 48, latitude: 24, longitude: 56)
Coordinates:
  * year       (year) int32 2000 2001 2002 2003 2004 ... 2014 2015 2016 2017
  * week       (week) int32 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48
  * latitude   (latitude) float64 80.5 79.75 79.0 78.25 ... 64.75 64.0 63.25
  * longitude  (longitude) float64 18.0 18.75 19.5 20.25 ... 57.75 58.5 59.25
Data variables:
    sic        (year, week, latitude, longitude) float64 0.07345 ... -1.0
Attributes:
    description:  Weekly Sea Ice Concentration

In [4]:
# get mask from ORAS4 data
mask_ocean = xr.where(sic_xr['sic'][0,0,:,:] < 0, 1, 0)
mask_ocean.rename('mask')

<xarray.DataArray 'mask' (latitude: 24, longitude: 56)>
array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 0, ..., 1, 1, 1]])
Coordinates:
    year       int32 2000
    week       int32 1
  * latitude   (latitude) float64 80.5 79.75 79.0 78.25 ... 64.75 64.0 63.25
  * longitude  (longitude) float64 18.0 18.75 19.5 20.25 ... 57.75 58.5 59.25

In [5]:
ohc_xr = xr.load_dataset(oras_path / "ohc_monthly_oras2erai_1978_2017.nc")
# the coordinate name is wrong for week, it should be month
ohc_xr = ohc_xr.rename({'week': 'month'})
# 1 more year inlucded to calculate the relative difference
ohc_xr = ohc_xr.sel(year=slice(start_year - 1, end_year), latitude=slice(81, 63),
 longitude=slice(18, 59.5))
ohc_xr

<xarray.Dataset>
Dimensions:    (year: 19, month: 12, latitude: 24, longitude: 56)
Coordinates:
  * year       (year) int32 1999 2000 2001 2002 2003 ... 2014 2015 2016 2017
  * month      (month) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * latitude   (latitude) float64 80.5 79.75 79.0 78.25 ... 64.75 64.0 63.25
  * longitude  (longitude) float64 18.0 18.75 19.5 20.25 ... 57.75 58.5 59.25
Data variables:
    OHC        (year, month, latitude, longitude) float64 -672.4 -672.4 ... 0.0
Attributes:
    description:  Monthly Ocean Heat Content regridded on rectilinear grid

In [6]:
# turn dataset into sequences
sic_seq = sic_xr.sic.values.reshape(-1, sic_xr.latitude.size, sic_xr.longitude.size)
ohc_seq_monthly = ohc_xr.OHC.values.reshape(-1, ohc_xr.latitude.size, ohc_xr.longitude.size)

In [7]:
# interpolation of ohc from monthly to weekly
ohc_diff_weekly = (ohc_seq_monthly[1:,:,:] - ohc_seq_monthly[:-1,:,:]) / 4
ohc_seq = np.zeros((sic_xr.year.size * nweeks, sic_xr.latitude.size, sic_xr.longitude.size),dtype=float)
# calculate relative difference, note that the first year data is edge case
for i in range(4):
    ohc_seq[3-i::4,:,:] = ohc_seq_monthly[12:,:,:] - i * ohc_diff_weekly[11:,:,:]

In [8]:
def normalizer(x):
    max_value = np.amax(x)
    min_value = np.amin(x)
    y = (x - min_value)/(max_value - min_value)
    return y

In [9]:
# normalize ohc
ohc_norm_seq = normalizer(ohc_seq)

Train/test data splitting.

In [10]:
# we deal with timeseries, the data should not be shuffled
train_years = 12

Build neural network.

In [11]:
class MetNet(nn.Module):
    def __init__(self, input_channels = 2, kernels=[2, 4, 1], dropout = 0.2):
        super().__init__()
        self.kernels = kernels
        # layer 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(input_channels, kernels[0], kernel_size=5, stride = 1, padding = 2),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        # layer 2
        self.layer2 = nn.Sequential(
            nn.Conv2d(kernels[0], kernels[1], kernel_size=5, stride = 1, padding = 2),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        # layer 3
        self.layer3 = nn.Sequential(
            nn.Conv2d(kernels[1], kernels[2], kernel_size=5, stride = 1, padding = 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        return x

Set up the environment for training.

In [12]:
print ("Pytorch version {}".format(torch.__version__))
#use_cuda = torch.cuda.is_available()
#print("Is CUDA available? {}".format(use_cuda))
input_channels = 2
epochs = 30
kernels = [2, 4, 1]
learning_rate = 0.01
dropout = 0.2

Pytorch version 1.10.2


In [13]:
# construct model
model = MetNet(input_channels, kernels, dropout)
# choose optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# loss function
loss_fn = torch.nn.MSELoss(reduction='mean')
# check the setups
print(model)
print(loss_fn)
print(optimizer)

MetNet(
  (layer1): Sequential(
    (0): Conv2d(2, 2, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
  )
  (layer2): Sequential(
    (0): Conv2d(2, 4, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
  )
  (layer3): Sequential(
    (0): Conv2d(4, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
  )
)
MSELoss()
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)


In [14]:
hist = np.zeros(epochs)
# switch model into train mode
model.train()
for epoch in range(epochs):
    running_loss = 0.0
    for timestep in range(train_years * nweeks):
        X_train = torch.tensor(np.stack((sic_seq[timestep,:,:],
                                        ohc_norm_seq[timestep,:,:]))
                              ).float().view(-1, input_channels, 
                                             sic_xr.latitude.size,
                                             sic_xr.longitude.size)
        x_var = torch.autograd.Variable(X_train)
        y_train = torch.tensor(sic_seq[timestep+1,:,:]).float().view(-1, 1, 
                                                                     sic_xr.latitude.size,
                                                                     sic_xr.longitude.size)
        y_var = torch.autograd.Variable(y_train)
        # forward pass
        outputs = model(x_var)
        # check loss
        loss = loss_fn(outputs, y_var)
        # zero the parameter gradients
        optimizer.zero_grad()
        # computing gradients
        loss.backward()
        # accumulating running loss
        running_loss += loss.item()
        # updated weights based on computed gradients
        optimizer.step()

    if epoch % 2 == 0:    
        print(f'Epoch {epoch + 1}/{epochs} running accumulative loss across all batches:' + 
              f'{running_loss:.3f}')
    running_loss = 0.0

Epoch 1/30 running accumulative loss across all batches:217.483
Epoch 3/30 running accumulative loss across all batches:215.844
Epoch 5/30 running accumulative loss across all batches:214.507
Epoch 7/30 running accumulative loss across all batches:213.708
Epoch 9/30 running accumulative loss across all batches:213.972
Epoch 11/30 running accumulative loss across all batches:213.096
Epoch 13/30 running accumulative loss across all batches:213.101
Epoch 15/30 running accumulative loss across all batches:213.053
Epoch 17/30 running accumulative loss across all batches:212.880
Epoch 19/30 running accumulative loss across all batches:212.473
Epoch 21/30 running accumulative loss across all batches:212.701
Epoch 23/30 running accumulative loss across all batches:212.978
Epoch 25/30 running accumulative loss across all batches:212.637
Epoch 27/30 running accumulative loss across all batches:212.929
Epoch 29/30 running accumulative loss across all batches:212.488


In [15]:
# save the general checkpoint
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss.item()
            }, Path('../xai/models/metnet_training_checkpoint.pt'))
print("The checkpoint of the model and training status is saved.")

The checkpoint of the model and training status is saved.
